In [ ]:

#Notes

#1.Run cells in the code from top to down smoothly to see its working
#2. Read The COMMENTS and change parameter accordinly
#

# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style="darkgrid")
from time import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
# Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA    

# Pretty display for notebooks
%matplotlib inline
from random import shuffle

In [ ]:
####################LPCA######CODE##### same # beta setting ########
               ### fixing alpha to 0 and (varying beta_avg) #########
    # In beta_avg config all sub groups have same acceptance rate 

#data1 is sensitive group in binary matrix sets 
import time
import pulp as p 
def min_sum_lpca(data1,beta_initial,eps,e,beta,alpha):
#here beta_avg=beta[0], same beta is used for ranked weighting
    import pulp as p 
    import math
    
    m=data1.shape[0]
    n=data1.shape[1]
    print('dimension of data')
    print(m,n)
    
    ################ sorted result
    a1=0
    a2=0
    b1=0
    b2=0
    c1=0
    c2=0
    d1=0
    d2=0
    e1=0
    e2=0
    h1=[]
    h2=[]
    h3=[]
    h4=[]
 
    key1=[]
    key2=[]
    key3=[]
    key4=[]

    cost=np.zeros(n,dtype=int)
    data2=np.zeros((m,n),dtype=int)
    for i in range(n):
        if data1[0][i]==1:            

            h1.append(e[i][1])
            key1.append(i)
            if data1[2][i]==1:
                a1=a1+1
            elif data1[3][i]==1:
                b1=b1+1
           

        elif data1[1][i]==1:
            h2.append(e[i][1])
            key2.append(i)
            if data1[2][i]==1:
                a2=a2+1
            elif data1[3][i]==1:
                b2=b2+1
            
            
        if data1[2][i]==1:
            h3.append(e[i][1])
            key3.append(i)
            
        elif data1[3][i]==1:
            h4.append(e[i][1])
            key4.append(i)
        

    for i in range(1,len(h1)):
        for j in range(i,0,-1):
            var=0
            var2=0
            if h1[j-1]<h1[j]:
                index=j
                var=h1[j]
                h1[j]=h1[j-1]
                h1[j-1]=var

                var2=key1[j]
                key1[j]=key1[j-1]
                key1[j-1]=var2
            else:
                break
    

    for i in range(1,len(h2)):
        for j in range(i,0,-1):
            var=0
            var2=0
            if h2[j-1]<h2[j]:
                index=j
                var=h2[j]
                h2[j]=h2[j-1]
                h2[j-1]=var

                var2=key2[j]
                key2[j]=key2[j-1]
                key2[j-1]=var2
            else:
                break
    for i in range(1,len(h3)):
        for j in range(i,0,-1):
            var=0
            var2=0
            if h3[j-1]<h3[j]:
                index=j
                var=h3[j]
                h3[j]=h3[j-1]
                h3[j-1]=var

                var2=key3[j]
                key3[j]=key3[j-1]
                key3[j-1]=var2
            else:
                break
    for i in range(1,len(h4)):
        for j in range(i,0,-1):
            var=0
            var2=0
            if h4[j-1]<h4[j]:
                index=j
                var=h4[j]
                h4[j]=h4[j-1]
                h4[j-1]=var

                var2=key4[j]
                key4[j]=key4[j-1]
                key4[j-1]=var2
            else:
                break
    #####alpha2 is just another weight influencing parameter for now its neutral with ["ones"] vector
    alpha2=[1,1,1,1]
    
#####Setup2#################LPCA with ranked sensitive groups with weight equalization    
    for j in range(len(key1)):    
               
        data2[0][key1[j]]=(j+1)*alpha2[0]
    for j in range(len(key2)):
        data2[1][key2[j]]=(j+1)*((beta[0]*len(key1))/(beta[1]*len(key2)))*alpha2[1]
    
    for j in range(len(key3)):
        data2[2][key3[j]]=(j+1)*alpha2[2]
                         
        
    for j in range(len(key4)):                            
        data2[3][key4[j]]=(j+1)*((beta[2]*len(key3))/(beta[3]*len(key4)))*alpha2[3]                         
 
 
#sum up the weighted subgroup rank in cost        
    for j in range(n):
        summ=0
        for i in range(m):
       
            summ=summ+data2[i][j] 
        cost[j]=summ
        
        
    ################
    
    
    Lp_prob = p.LpProblem('Problem', p.LpMinimize)  
    ###############################Optimization fuction for LPCA###################
# beta_avg(convergence point to achieve least DP at alpha=0   
# beta_initial (acceptance rate) config obtaind for each group from random forest prediction)   


    X=np.zeros(n+m+1,dtype=p.LpVariable)
    Y=np.zeros(m,dtype=p.LpVariable)
    
    sizes=np.zeros(m,dtype=int)
  
    max_size=0
    for i in range(m):
        count=0
        for j in range(n):
            if data1[i][j]==1:
                count=count+1 
        if count>max_size:
            max_size=count
        sizes[i]=count
    print(sizes)        
    
 ################ As all beta[i]'s are same beta_avg=beta[0] #################   
    
    beta_avg=beta[0]
    
    
    
    select_sizes=np.zeros(m,dtype=int)
   
    size_final=np.zeros(m,dtype=int)

    for i in range(m):
        var1 = str(n+100+i)
        Y[i]=p.LpVariable(var1,lowBound=0,upBound=1,cat='Continuous')
    
    for i in range(n):
        var1=str(i)       
        X[i]=p.LpVariable(var1,lowBound=0,upBound=1,cat='Integer')
   
    X[n]=p.LpVariable(str(n),lowBound=0,upBound=1,cat='Continuous')  
#minimize Objective#
    Lp_prob+= p.lpSum([(X[j])*cost[j] for j in range(n)])
    

    for i in range(2*m):
        if i<m:

            Lp_prob += p.lpSum([(X[j])*(data1[i][j]) for j in range(n)]) >= (Y[i]-eps)*sizes[i]
            Lp_prob += p.lpSum([(X[j])*(data1[i][j]) for j in range(n)]) <= (Y[i]+eps)*sizes[i]
    
    '''
    
# for minimum ddp at alpa=0 setup (like paper) change (1- alpha) to (alpha) & (alpha) to (1-alpha)    
    for i in range(m):
            if beta_initial[i] >= beta_avg:

                Lp_prob += Y[i] >= (1-alpha)*beta_initial[i] +alpha*beta_avg
                Lp_prob += Y[i] <= (1-alpha)*beta_initial[i] +alpha*beta_avg
               
            else:
                Lp_prob += Y[i] >= (1-alpha)*beta_initial[i] + alpha*beta_avg
                Lp_prob += Y[i] <= beta_avg   
    '''      


    for i in range(m):
        Lp_prob += Y[i] >= (alpha)*beta_initial[i] +(1-alpha)*beta_avg
        Lp_prob += Y[i] <= (alpha)*beta_initial[i] +(1-alpha)*beta_avg
                         
   
    #####################################
    status = Lp_prob.solve()   # Solver 
    print(p.LpStatus[status]) 
    print("objective is:")        
    print(p.value(Lp_prob.objective))
    print("discripency is:") 
    print(p.value(X[n]))
    x=np.zeros(n,dtype=float)

   # The solution status 
    Synth1={}
    Synth2={}
    # # Printing the final solution 
    for i in range(n):
        if(p.value(X[i])==1):
            Synth1[i]=1 
            Synth2[i]=-1
#             if(data1[2][i]==1):
#                 print("no")
        else:
            Synth1[i]=-1
            Synth2[i]=1
    Synthu1=Synth1  
    Synthu2=Synth2  
    
              
    return Synthu1,Synthu2   


In [ ]:
#rf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%pylab inline
from random import *
from subprocess import check_output
def german_rf(X,Y):
    #Split data into training and test datasets (training will be based on 70% of data)
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10,shuffle=True)     #test_size: if integer, number of examples into test dataset; if between 0.0 and 1.0, means proportion
    print('There are {} samples in the training set and {} samples in the test set'.format(X_train.shape[0], X_test.shape[0]))

    
    #Scaling data
    #from sklearn.preprocessing import StandardScaler
    
    from sklearn.model_selection import cross_val_score
    from sklearn.ensemble import RandomForestClassifier
    #sc = StandardScaler(with_mean=False)
    
    
    #sc.fit(X_train)
    #X_train_std = sc.transform(X_train)
    #X_test_std = sc.transform(X_test)

    #X_train_std and X_test_std are the scaled datasets to be used in algorithms

    #Applying SVC (Support Vector Classification)
#     from sklearn.svm import SVC
#     svm = SVC(kernel='rbf', random_state=0, gamma=.1, C=10.0,probability=True)
    
    rf = RandomForestClassifier(n_estimators=99, max_depth=None, min_samples_split=15, random_state=0)
    
    print(Y_train.dtypes)
    Y_train=Y_train.astype('int')
    print(Y_train.dtypes)
    
    print(Y_test.dtypes)
    Y_test=Y_test.astype('int')
    print(Y_test.dtypes)
    
    
    rf.fit(X_train, Y_train)
    print('The accuracy of the RF classifier on training data is {:.2f}'.format(rf.score(X_train, Y_train)))
    print('The accuracy of the RF classifier on test data is {:.2f}'.format(rf.score(X_test, Y_test)))
    print('####Train prediction Label###############################################')
    Y_train_pred=rf.predict(X_train)
    #print(y_1)
    Y_test_pred=rf.predict(X_test)

    print('####Actual Train Label###############################################')


    print('####Change to colors###############################################')
        
    e=rf.predict_proba(X_test)
    print(e)
    return X_test,Y_test_pred,Y_test,e
     



In [ ]:

def main(datax, y_test, y_test_pred,e): 
        
    n=datax.shape[1]
    s=datax.shape[0]    
    data = np.zeros((s, n), dtype = int)
    
    r = np.zeros(n, dtype = int) 
    
    for i in range(n):
        if int(y_test.iloc[i])==1 :
            r[i]=1
        else :
            r[i]= -1  
    
    r2 = np.zeros(n, dtype = int) 
    for i in range(n):
        if int(y_test_pred[i])==1 :
            r2[i]=1
        else :
            r2[i]= -1          
    ar=[]
    
    for j in range(s):
        print("sensitive attribute ",(j+1)) 
        a=0
        b=0
        acc1=0
        acc2=0
        for i in range(n):
                data[j][i]= datax.iloc[j,i]
                if data[j][i]== 1 :
                    a=a+1
                    if r[i]==1:
                         acc1=acc1+1 

        print("ACTUAL----------total ,accepted, aceeptance rate:")             
        a1=float(acc1/a)
        print(a)
        
        print(acc1)
        print(a1)
        ar.append(a1)
    maxi= max(ar)
    mini= min(ar)
    DP=float(maxi-mini)
    print("data acceptance rates")
    print(ar)
    print("data DP")
    print(DP)
    
    ar=[]
    
    for j in range(s):
        print("sensitive attribute ",(j+1)) 
        a=0
        b=0
        acc1=0
        acc2=0
        prec=0
        reca=0
        accur=0
        FP=0
        FN=0
        TP=0
        TN=0
        for i in range(n):
             if data[j][i]== 1 :
                    a=a+1
                    if r2[i]==1:
                        acc1=acc1+1 
                        if r[i]==1:
                            TP=TP+1
                        else:
                             FP=FP+1                
                    else:
                        if r[i]==1:
                            FN=FN+1
                        else:
                            TN=TN+1    
        
        print("prec reca accuracy for each sens") 
        prec= float(TP/(TP+FP))
        reca= float(TP/(TP+FN))
        accur= float((TP+TN)/a)
        print(prec,reca,accur)
        
        print("Random Forest---------total , accepted, aceeptance rate:")             
        
        a1=float(acc1/a)
        print(a)
        
        print(acc1)
        print(a1)
        ar.append(a1)
#COMMENT   : "ar" #############use this predicted RandomForet accptance rate(config) hardcoded or
#can pass it through LPCA as lpca() 
#above ar: Random forest acceptance rate is use as beta_initial/beta (named as beata_initial in paper)
######################################RF acceptance rate as beta_initial #############   
    beta_initial=ar

########################################################################################    
 
    
    maxi= max(ar)
    mini= min(ar)
    DP=float(maxi-mini)
    
    print("data acceptance rates")
    print(ar)
    
    print("data DP")
    print(DP) 
    
    print("Random Forest accuracy--------------------------")
    prec=0
    reca=0
    accur=0
    FP=0
    FN=0
    TP=0
    TN=0
    for i in range(n):
            if r2[i]==1:
                acc1=acc1+1 
                if r[i]==1:
                    TP=TP+1
                else:
                     FP=FP+1                
            else:
                if r[i]==1:
                     FN=FN+1
                else:
                     TN=TN+1    

        
    prec= float(TP/(TP+FP))
    reca= float(TP/(TP+FN))
    accur= float((TP+TN)/n)
    print(prec,reca,accur)
    
    
    
    fi= np.zeros(n,dtype=int) 
#(beta_converge, alpha and epsilon are the parameters of LPCA ) 
# alpha=1 initial predicted(beta configs) by classifier (RF), at alpha=0 Least DDP config at beta_converge

#Example for beta_converge where at all cases alpha=0
   
    

    beta_converge=[.68,.7,.75,.8,.84,.87,.89 ,.91]
    
    gamma=np.zeros((8,4),dtype=float)
    for i in range(4):
        for j in range(8):
            gamma[j][i]=beta_converge[j]
    alpha=[0]
    epsilon=[.01,.005]
    t=0
    new=0
    
    
    t=0
    a=0
    
    for eps in epsilon:
        for new in range(8):
        
            for a in alpha:
                k=0
                u1,u2=min_sum_lpca(data,beta_initial,eps,e,gamma[new],a)
                #######################Disp_impact#######################  
                print("gamma-epsilon-delta",gamma[new],eps)
                accu_all=[]
                DP_all=[]
                precision_all=[]
                recall_all=[]
                ar_all=[]
                acceptance_rate=np.zeros((7,28),dtype=float)
                count=0
                print("<--------------------------------------->")
                print("iteration t",k)
                k=k+1


                for i in range(n):
                     fi[i] = u1[i]


                for j in range(s):
                    print("sensitive attribute ",(j+1)) 

                    TP=0
                    FP=0
                    FN=0
                    TN=0
                    precision=0
                    recall=0
                    for i in range(n):
                         if data[j][i]== 1 :                        
                            if fi[i]==1 and r[i]==1:
                                TP=TP+1
                            if fi[i]==1 and r[i]==-1:
                                FP=FP+1 
                            if fi[i]==-1 and r[i]==1:
                                FN=FN+1
                            if fi[i]==-1 and r[i]==-1:
                                TN=TN+1    
                    if TP+FP !=0:
                        precision=float(TP/(TP+FP))
                    #print("precision",precision)
                    if TP+FN !=0:    
                        recall=float(TP/(TP+FN))
                    #print("recall",recall)

                    precision_all.append(precision)
                    recall_all.append(recall)
                    #print("TP,FP,TN,FN")
                    #print(TP,FP,TN,FN)

                    a=0
                    b=0
                    acc1=0
                    acc2=0
                    for i in range(n):
                            if data[j][i]== 1 :
                                a=a+1
                                if fi[i]==1:
                                     acc1=acc1+1 

        #                         print("total ,fair accepted, aceeptance rate:")             
                    a1=float(acc1/a)



        #                         print(a)
        #                         print(acc1)
        #                         print(a1)
                    ar_all.append(a1)

                count = count+1
                maxi=max(ar_all)
                mini= min(ar_all)
                DP=float(maxi-mini)
                print("individual acceptance rates")
                print(ar_all)
                print("individual precision")
                print(precision_all)
                print("individual recall")
                print(recall_all)
                print("DP all")
                print(DP)
                f_acc=0
                for i in range(n):
                     if fi[i] == r[i]:
                            f_acc=f_acc+1
                f_acc_l=float((f_acc*100)/n) 

        #######################################################################33   

        #                         print("sensitive attribute ",(j+1)) 





                TP=0
                FP=0
                FN=0
                TN=0
                precision=0
                recall=0
                accu=0
                for i in range(n):
                        if fi[i]==1 and r[i]==1:
                            TP=TP+1
                        if fi[i]==1 and r[i]==-1:
                            FP=FP+1 
                        if fi[i]==-1 and r[i]==1:
                            FN=FN+1
                        if fi[i]==-1 and r[i]==-1:
                            TN=TN+1    

                if TP+FP!=0:
                    precision=float(TP/(TP+FP))
                print("precision all",precision)
                if TP+FN!=0:
                    recall=float(TP/(TP+FN))


                print("recall all",recall)
                accu=float((TP+TN)/(TP+FN+TN+FP))


                print("accuracy all",accu)



                print("TP,FP,TN,FN")
                print(TP,FP,TN,FN)
        #                         print("total ,fair accepted, aceeptance rate:")             
                a1=float(acc1/a)


    print("<--------------------------------------->")
    alpha_weight=np.arange(0,1.05,.05)        
    return accu_all,DP_all,acceptance_rate,alpha_weight

In [ ]:

import time
# import pulp as p 
# from random import *
from sklearn import preprocessing
# Add column names to data set

data= pd.read_csv('data/german.csv' , skipinitialspace=True)
print(data.head())
print(data.shape[0],data.shape[1])

#sensitive columns name '12'='age','8'='gender/personal_status'  '19'- foreign workers 20'=1(good)/2(bad))

# print(sens)
r=data[['20']]
# print(r)
p=data.shape[0]
for i in range(0,p):  
    if data.loc[i,"12"]>25 :
               data.loc[i,"12"] = 1 
    else :
               data.loc[i,"12"] = 2
    if r.loc[i,'20'] == 1 :
               r.loc[i,"20"] = 1 
    else: 
               r.loc[i,"20"] = 0  
            
print(data['20'].value_counts())            

print(data['8'].value_counts())
print(data['12'].value_counts())
print(data['19'].value_counts())

####################################################################################


# Initialize a scaler, then apply it to the features
'''
scaler = MinMaxScaler() # default=(0, 1)
num_col = dat.dtypes[dat.dtypes != 'object'].index
features_log_minmax_transform = pd.DataFrame(data = dat)
features_log_minmax_transform[num_col] = scaler.fit_transform(features_log_minmax_transform[num_col])


display(features_log_minmax_transform.head())
'''

# sens=DATA[['sex','race']]
#Data_c = pd.get_dummies(features_log_minmax_transform, columns=['sex','race','workclass','education','marital-status','occupation','relationship','native-country'], prefix =['s','r','work','edu','ms','occ','rls','nc'])
m=data.shape[1]
data_c1=data.iloc[:,0:m-1]
min_max_scaler = preprocessing.MinMaxScaler()
data_c2 = min_max_scaler.fit_transform(data_c1)
data_c = pd.DataFrame(data_c2,columns=data_c1.columns)
print(data_c)



X_test,Y_test_pred,Y_test,e = german_rf(data_c , r)
#print(X_test.iloc[:,:])
X_test.reset_index(drop=True, inplace=True)
# Y_test_pred.reset_index()
Y_test.reset_index(drop=True, inplace=True)

# print(X_test)
# print(Y_test_pred)
# print(Y_test)
sens=X_test[['8', '12' ]]
# print(sens)

    


     
# for i in range(0,p):  
#     if r.loc[i,'y'] == 1 :
#                r.loc[i,"y"] = 1 
#     else: 
#                r.loc[i,"y"] = 0 
print(sens['8'].value_counts())            
print(sens['12'].value_counts())
#print(sens['19'].value_counts())
sens1=pd.get_dummies(sens, columns=['8','12'], prefix =['8','12'])
sensitive=sens1.T
print(sensitive) 




# 
'''

'''
# beta=.70
# .0075, .7533

# beta=.72
# .0048, .76





In [ ]:
###################################Run and observe the results at varius parameters
# Run for getting the Final output

accu_all,DP_all,acceptance_rate,alpha_weight = main(sensitive, Y_test, Y_test_pred,e )

# Observe group wise acc_rate,precision, recall, DP
#eps=.005 ensures DDP<.01  , configuration within [beta_avg+eps ,beta_avg-eps] 
#see the the outputs at alpha =0 beta_avg varies
#SEE Optimal group wise acceptance rate/config (individual beta)

#optimal before iterations tells the valid configuration on parameters.
#undefined ,means invalid configuration on given parameters
       #if undefined increase eps from .005 to .01 
#gamma is beta_avg in vector form    

In [ ]:
#Result

In [ ]:
#beta same from LPCA

acc_rate=[
[0.6910994764397905, 0.6972477064220184, 0.6938775510204082, 0.6909090909090909],
[0.743455497382199, 0.7431192660550459, 0.7428571428571429, 0.7454545454545455],
[0.7905759162303665, 0.7981651376146789, 0.7918367346938775, 0.8],
[0.8324607329842932, 0.8348623853211009, 0.8326530612244898, 0.8363636363636363],
    [0.8638743455497382, 0.8623853211009175, 0.8612244897959184, 0.8727272727272727],

[0.900523560209424, 0.908256880733945, 0.9020408163265307, 0.9090909090909091]
]



prec=[
    
[0.8409090909090909, 0.7368421052631579, 0.8352941176470589, 0.6578947368421053],
[0.8380281690140845, 0.7283950617283951, 0.8241758241758241, 0.6829268292682927],
[0.8211920529801324, 0.6896551724137931, 0.8041237113402062, 0.6363636363636364],
[0.8113207547169812, 0.6703296703296703, 0.7892156862745098, 0.6304347826086957],
  [0.806060606060606, 0.648936170212766, 0.7819905213270142, 0.6041666666666666],

[0.7965116279069767, 0.6262626262626263, 0.7647058823529411, 0.6]
]



rec=[
    
[0.7928571428571428, 0.8888888888888888, 0.8208092485549133, 0.8333333333333334],
[0.85, 0.9365079365079365, 0.8670520231213873, 0.9333333333333333],
[0.8857142857142857, 0.9523809523809523, 0.9017341040462428, 0.9333333333333333],
[0.9214285714285714, 0.9682539682539683, 0.930635838150289, 0.9666666666666667],
   [0.95, 0.9682539682539683, 0.953757225433526, 0.9666666666666667],

[0.9785714285714285, 0.9841269841269841, 0.976878612716763, 1.0]
]

accu= [0.7433,0.76666,0.75666,0.7566,0.7533,0.7466]

beta=[.7,.75,.8,.84,.87,.91]

beta_p=[]
beta_r=[]
beta_check=[0.8795811518324608, 0.7614678899082569, 0.8571428571428571, 0.7454545454545455]
import numpy as np


   
   
    
#P R P R P R R 
#0.7251,0.8101 0.5470,0.7329 0.6311,0.7389 0.5308,0.6172 0.6764,0.4776  0.4666,0.5333  0.2666,0.7333
#      
weighted_precision=[]
weighted_recall=[]
#p=[]
#r=[0,2,4,1,3,5,6]

dp_list=[]
sizes=[191,109,245,55]
for i in range(6):
    weight_prec=0
    weight_p=0
    weight_rec=0
    weight_r=0
    cnt1=0
    cnt2=0
    for j in range(4):
        #print(j)
              
        
        if beta[i] <=beta_check[j]:
            weight_prec=weight_prec+sizes[j]*prec[i][j]
            weight_p=weight_p+sizes[j]
            cnt1=1
        else:  
            weight_rec=weight_rec+sizes[j]*rec[i][j]
            weight_r=weight_r+sizes[j]
            cnt2=1
    if cnt1==1:
        wp=weight_prec/weight_p
        weighted_precision.append(wp)
        beta_p.append(beta[i])

    if cnt2==1: 
        wr=weight_rec/weight_r
        weighted_recall.append(wr) 
        beta_r.append(beta[i])
            
   
    
    
len1=(len(weighted_precision)) 
len2=(len(weighted_recall)) 
    
print(weighted_precision, weighted_recall,beta_p,beta_r,len1,len2)
'''
[0.9520125938773091, 0.9403591343418822, 0.8608764055619338],0.8561212064210523, 0.8128034718440105, 0.782912341254576, 0.7358009995069338
0.4610404591572201, 0.5551300667433751, 0.5983458776930215, 0.6580824258162613,0.6476895465902177, 0.7070185074558867, 0.754950250161771
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
np.set_printoptions(precision=4)  # For compact display.
plt.style.use('seaborn-white')


'''
y1=savgol_filter(a, 6, 2)
y2=savgol_filter(b, 6, 2)
y3=savgol_filter(c, 6, 2)
y4=savgol_filter(d, 6, 2)
y5=savgol_filter(e, 6, 2)
y6=savgol_filter(f, 6, 2)
y7=savgol_filter(g, 6, 2)
'''



fig = plt.figure()
ax = plt.subplot(111)
# weighted_precision=[0.8319821845141085, 0.8289612688936687, 0.8158017367577965, 0.8190048909330003, 0.8046384596813133, 0.7804878048780488] 
# weighted_recall= [0.7666666666666667, 0.8666666666666668, 0.8708865660085173, 0.9248935346496321, 0.927902973395931, 0.95113420956051]
beta1=beta_p
beta2=beta_r
 
# accu=[0.6533,0.7033,0.7466, 0.7366,0.7433,0.74,0.7333]
'''
weighted_precision=[0.9520125938773091, 0.9403591343418822, 0.8608764055619338,0.8561212064210523, 0.8128034718440105, 0.782912341254576, 0.7358009995069338]
weighted_recall=[0.4610404591572201, 0.5551300667433751, 0.5983458776930215, 0.6580824258162613,0.6476895465902177, 0.7070185074558867, 0.754950250161771]
beta1=[.02,.025,.05,.1,.14,.16,.2]
beta2=[.1,.14,.16,.2,.25,.3,.35]

accu=[ 0.7774,0.7813,0.7954,0.8157,0.82442,0.8245,0.8216,0.8146,0.7954,0.7699]
'''
ax.plot(beta1,weighted_precision,label='Weighted Precision',color='blue',marker='^',linestyle='--')  
ax.plot(beta2,weighted_recall,label='Weighted Recall',color='cyan',marker='^',linestyle='--')
ax.plot(beta,accu,label=' Accuracy',color='red',marker='^',linestyle='--')
#ax.vlines(y=[.1992], ymin=[0], ymax=[1], colors='purple', linestyles='--', lw=2, label='PRedict avg. acc.')
#plt.axvline(.1992, color='green', linestyle='--')
#plt.axvline(.10, color='orange', linestyle='--')
#plt.axvline(.20, color='orange', linestyle='--')
plt.axvline(0.8795, color='orange', linestyle='--')
plt.axvline(0.7614, color='orange', linestyle='--')
plt.axvline(0.8571, color='orange', linestyle='--')
plt.axvline(0.7454, color='orange', linestyle='--')

#  [0.8795811518324608, 0.7614678899082569, 0.8571428571428571, 0.7454545454545455]

plt.title('')
ax.set_xlabel('Acceptance Rate')
ax.set_ylabel('Performance Metrics') 
# ax.set_ylabel('% in +ve class (Acceptance Rate)') 

ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.30), shadow=True, ncol=10)
plt.show() 
fig.savefig('a2.png') 

In [ ]:
accu_all,DP_all,acceptance_rate,alpha_weight = main(sensitive, Y_test, Y_test_pred ,e)

In [ ]:
[0.2576267506242536, 0.07575757575757576, 0.2111662955985614, 0.08934426229508197, 0.2698019801980198, 0.1, 0.034782608695652174]
[0.27979274611398963, 0.0718288334182374, 0.07343212490076739, 0.10702734489855925, 0.06472727272727273, 0.0]
[0.26282051282051283, 0.58125, 0.5182539682539683, 0.21296296296296297]
[0.8795811518324608, 0.7614678899082569, 0.8571428571428571, 0.7454545454545455]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
np.set_printoptions(precision=4)  # For compact display.
plt.style.use('seaborn-white')


'''
y1=savgol_filter(a, 6, 2)
y2=savgol_filter(b, 6, 2)
y3=savgol_filter(c, 6, 2)
y4=savgol_filter(d, 6, 2)
y5=savgol_filter(e, 6, 2)
y6=savgol_filter(f, 6, 2)
y7=savgol_filter(g, 6, 2)
'''



fig = plt.figure()
ax = plt.subplot(111)
weighted_precision=[0.8319821845141085, 0.8289612688936687, 0.8158017367577965, 0.8190048909330003, 0.8046384596813133, 0.7804878048780488] 
weighted_recall= [0.7666666666666667, 0.8666666666666668, 0.8708865660085173, 0.9248935346496321, 0.927902973395931, 0.95113420956051]
beta1=[.5,.6,.7,.75,.8,.84]
beta2=[.6,.7,.75,.8,.84,.88]
 
accu=[0.6533,0.7033,0.7466, 0.7366,0.7433,0.74,0.7333]
'''
weighted_precision=[0.9520125938773091, 0.9403591343418822, 0.8608764055619338,0.8561212064210523, 0.8128034718440105, 0.782912341254576, 0.7358009995069338]
weighted_recall=[0.4610404591572201, 0.5551300667433751, 0.5983458776930215, 0.6580824258162613,0.6476895465902177, 0.7070185074558867, 0.754950250161771]
beta1=[.02,.025,.05,.1,.14,.16,.2]
beta2=[.1,.14,.16,.2,.25,.3,.35]

accu=[ 0.7774,0.7813,0.7954,0.8157,0.82442,0.8245,0.8216,0.8146,0.7954,0.7699]
'''
ax.plot(beta1,weighted_precision,label='Weighted Precision',color='blue',marker='^',linestyle='--')  
ax.plot(beta2,weighted_recall,label='Weighted Recall',color='cyan',marker='^',linestyle='--')
ax.plot(beta,accu,label=' Accuracy',color='red',marker='^',linestyle='--')
#ax.vlines(y=[.1992], ymin=[0], ymax=[1], colors='purple', linestyles='--', lw=2, label='PRedict avg. acc.')
#plt.axvline(.1992, color='green', linestyle='--')
#plt.axvline(.10, color='orange', linestyle='--')
#plt.axvline(.20, color='orange', linestyle='--')
plt.axvline(0.8795, color='orange', linestyle='--')
plt.axvline(0.7614, color='orange', linestyle='--')
plt.axvline(0.8571, color='orange', linestyle='--')
plt.axvline(0.7454, color='orange', linestyle='--')

 
plt.title('')
ax.set_xlabel('Acceptance Rate')
ax.set_ylabel('Performance Metrics') 
# ax.set_ylabel('% in +ve class (Acceptance Rate)') 

ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.30), shadow=True, ncol=10)
plt.show() 
fig.savefig('a2.png') 

In [ ]:
prec=[[0.8269230769230769, 0.7160493827160493, 0.803921568627451, 0.696969696969697],
[0.8176100628930818, 0.7037037037037037, 0.8029556650246306, 0.6486486486486487],
[0.8098159509202454, 0.7108433734939759, 0.81, 0.6304347826086957],
[0.8098159509202454, 0.6976744186046512, 0.801980198019802, 0.6382978723404256]]

rec=[[0.9214285714285714, 0.9206349206349206, 0.9479768786127167, 0.7666666666666667],
[0.9285714285714286, 0.9047619047619048, 0.9421965317919075, 0.8],
[0.9428571428571428, 0.9365079365079365, 0.9364161849710982, 0.9666666666666667],
[0.9428571428571428, 0.9523809523809523, 0.9364161849710982, 1.0]]

accu=[0.78,0.77, 0.7766, 0.7733]

In [ ]:
#yang ours

gamma=[[0.8196, 0.7512, 0.8424, 0.6222],
[0.8324, 0.7431, 0.8285, 0.6727],
[0.8534, 0.7614, 0.8163, 0.8363],
[0.8534, 0.7889, 0.8244, 0.8545]]

prec=[[0.8269230769230769, 0.7160493827160493, 0.803921568627451, 0.696969696969697],
[0.8176100628930818, 0.7037037037037037, 0.8029556650246306, 0.6486486486486487],
[0.8098159509202454, 0.7108433734939759, 0.81, 0.6304347826086957],
[0.8098159509202454, 0.6976744186046512, 0.801980198019802, 0.6382978723404256]]

rec=[[0.9214285714285714, 0.9206349206349206, 0.9479768786127167, 0.7666666666666667],
[0.9285714285714286, 0.9047619047619048, 0.9421965317919075, 0.8],
[0.9428571428571428, 0.9365079365079365, 0.9364161849710982, 0.9666666666666667],
[0.9428571428571428, 0.9523809523809523, 0.9364161849710982, 1.0]]

accu=[0.78,0.77, 0.7766, 0.7733]

weighted_precision=[]
weighted_recall=[]
p=[2]
r=[0,1,3]
#print(np.transpose(acc_rate))
weight_prec=0
weight_p=0
weight_rec=0
weight_r=0
sizes=[191,109,245,55]
dp_list=[]

for i in range(4):
    weight_prec=0
    weight_rec=0
    weight_p=0
    weight_r=0
    acc_list=[]
    for j in range(4):
        #print(j)
        if j in p:
            weight_prec=weight_prec+sizes[j]*prec[i][j]
            weight_p=weight_p+sizes[j]
            #print(j)
        if j in r:    
            weight_rec=weight_rec+sizes[j]*rec[i][j]
            weight_r=weight_r+sizes[j]
        #print(acc_rate[i][j])    
        #acc_list.append(acc_rate[i][j])
    #print(acc_list)
    #dp=max(acc_list)-min(acc_list)   
    #dp_list.append(dp)     
    wp=weight_prec/weight_p
    wr=weight_rec/weight_r
    weighted_precision.append(wp)
    weighted_recall.append(wr)
print( weighted_precision,weighted_recall,accu)
weight_prec1=weighted_precision
weight_rec1=weighted_recall
accu1=accu

#yang actual

gamma=[[0.8196, 0.7512, 0.8424, 0.6222],
[0.8324, 0.7431, 0.8285, 0.6727],
[0.8534, 0.7614, 0.8163, 0.8363],
[0.8534, 0.7889, 0.8244, 0.8545]]

prec=[[0.7995, 0.8145, 0.8151, 0.7380],
[0.8113, 0.6913, 0.7783, 0.7297],
[0.7914, 0.6746, 0.785, 0.6086],
[0.7914, 0.6627, 0.7772, 0.6170]]

rec=[[0.9108, 0.8913, 0.9304, 0.775],
[0.9214, 0.8888, 0.9132, 0.9],
[0.9214, 0.8888, 0.9075, 0.9333],
[0.9214, 0.9047, 0.9075, 0.9666]]



accu=[ 0.7757,0.7566,0.7366,0.7333]

dp=[0.2202,0.1597,0.0919,0.0655]



weighted_precision=[]
weighted_recall=[]
p=[2]
r=[0,1,3]
#print(np.transpose(acc_rate))
weight_prec=0
weight_p=0
weight_rec=0
weight_r=0
sizes=[191,109,245,55]
dp_list=[]

for i in range(4):
    weight_prec=0
    weight_rec=0
    weight_p=0
    weight_r=0
    acc_list=[]
    for j in range(4):
        #print(j)
        if j in p:
            weight_prec=weight_prec+sizes[j]*prec[i][j]
            weight_p=weight_p+sizes[j]
            #print(j)
        if j in r:    
            weight_rec=weight_rec+sizes[j]*rec[i][j]
            weight_r=weight_r+sizes[j]
        #print(acc_rate[i][j])    
        #acc_list.append(acc_rate[i][j])
    #print(acc_list)
    #dp=max(acc_list)-min(acc_list)   
    #dp_list.append(dp)     
    wp=weight_prec/weight_p
    wr=weight_rec/weight_r
    weighted_precision.append(wp)
    weighted_recall.append(wr)
print( weighted_precision,weighted_recall,accu)
weight_prec2=weighted_precision
weight_rec2=weighted_recall
accu2=accu

In [ ]:
p=[]
r=[]
a=[]
for i in range(4):
    p.append(weight_prec2[i])
    p.append(weight_prec1[i])
    r.append(weight_rec2[i])
    r.append(weight_rec1[i])
    a.append(accu2[i])
    a.append(accu1[i])

print(p)
print(r)
print(a)

In [ ]:
[0.8151, 0.803921568627451, 0.7783, 0.8029556650246306, 0.785, 0.81, 0.7772, 0.801980198019802]
[0.8837732394366198, 0.8972076905879722, 0.9080749295774648, 0.9013413816230718, 0.9132340845070422, 0.9445964676950592, 0.9232752112676056, 0.9546344735077129]
[0.7757, 0.78, 0.7566, 0.77, 0.7366, 0.7766, 0.7333, 0.7733]

In [ ]:
X_test,Y_test_pred,Y_test,e = law_school_svm(data_c , r)
#print(X_test.iloc[:,:])

X_test.reset_index(drop=True, inplace=True)
# Y_test_pred.reset_index()
Y_test.reset_index(drop=True, inplace=True)

# print(X_test)
# print(Y_test_pred)
# print(Y_test)
sens=X_test[['sex', 'race' ]]
# print(sens)

    


     
# for i in range(0,p):  
#     if r.loc[i,'y'] == 1 :
#                r.loc[i,"y"] = 1 
#     else: 
#                r.loc[i,"y"] = 0 
print(sens['sex'].value_counts())
print(sens['race'].value_counts()) 

#print(sens['19'].value_counts())
sens1=pd.get_dummies(sens, columns=['sex','race'], prefix =['s','r'])
sensitive=sens1.T
print(sensitive) 


accu_all,DP_all,acceptance_rate,alpha_weight = main(sensitive, Y_test, Y_test_pred,e )


In [ ]:

'''
Beta equals
beta=[[0.4973821989528796, 0.4954128440366973, 0.49795918367346936, 0.4909090909090909],
[0.5968586387434555, 0.6055045871559633, 0.6, 0.6],
[0.6963350785340314, 0.6972477064220184, 0.6979591836734694, 0.6909090909090909],
[0.7486910994764397, 0.7431192660550459, 0.746938775510204, 0.7454545454545455],
[0.7958115183246073, 0.8073394495412844, 0.8, 0.8],
[0.837696335078534, 0.8348623853211009, 0.8367346938775511, 0.8363636363636363],
[0.8795811518324608, 0.8715596330275229, 0.8775510204081632, 0.8727272727272727]]
precision=[0.8842105263157894, 0.7407407407407407, 0.860655737704918, 0.7037037037037037],
[0.8596491228070176, 0.7424242424242424, 0.8435374149659864, 0.696969696969697],
[0.8571428571428571, 0.7105263157894737, 0.8304093567251462, 0.6842105263157895],
[0.8321678321678322, 0.6790123456790124, 0.8087431693989071, 0.6341463414634146],
[0.8223684210526315, 0.6590909090909091, 0.7908163265306123, 0.6363636363636364],
[0.80625, 0.6483516483516484, 0.7804878048780488, 0.6086956521739131],
[0.7916666666666666, 0.631578947368421, 0.7627906976744186, 0.6041666666666666]]

recall=[[0.6, 0.6349206349206349, 0.6069364161849711, 0.6333333333333333],
[0.7, 0.7777777777777778, 0.7167630057803468, 0.7666666666666667],
[0.8142857142857143, 0.8571428571428571, 0.8208092485549133, 0.8666666666666667],
[0.85, 0.873015873015873, 0.8554913294797688, 0.8666666666666667],
[0.8928571428571429, 0.9206349206349206, 0.8959537572254336, 0.9333333333333333],
[0.9214285714285714, 0.9365079365079365, 0.9248554913294798, 0.9333333333333333],
[0.95, 0.9523809523809523, 0.9479768786127167, 0.9666666666666667]]

accu[0.6533,0.7033,0.7466, 0.7366,0.7433,0.74,0.7333]
'''

In [ ]:
#bilal
'''
gamma=[ [0.8324 ,0.688  ,0.8204 ,0.6   ],
       [0.8429 ,0.7339 ,0.7918 ,0.8545]
 [0.8376 ,0.7339 ,0.8163 ,0.7272],
 [0.8272 ,0.7431 ,0.7959 ,0.8   ] ]
prec=[[0.8050,0.7333,0.7860,0.7575],
[0.7888,0.6875,0.7886,0.6170],
[0.8062,0.7125,0.79,0.7],
[0.7974,0.6913,0.7897, 0.6363]]

rec=[[0.9142,0.8730,0.9132,0.8333],
[0.9071,0.8730,0.8843,0.9666],
[0.9214,0.9047,0.9132,0.9333],
[0.9,0.8888,0.8901,0.9333]]

dp=[  0.2324,0.1206, 0.1104 ,0.0841]
begin
[0.7633,0.7333, 0.7633, 0.74]
'''

In [ ]:
#yang 1>
'''
------------------Results  start----------------------------
prec=[[0.7995, 0.8145, 0.8151, 0.7380],
[0.8113, 0.6913, 0.7783, 0.7297],
[0.7914, 0.6746, 0.785, 0.6086],
[0.7914, 0.6627, 0.7772, 0.6170]]

rec=[[0.9108, 0.8913, 0.9304, 0.775],
[0.9214, 0.8888, 0.9132, 0.9],
[0.9214, 0.8888, 0.9075, 0.9333],
[0.9214, 0.9047, 0.9075, 0.9666]]

beta=[[0.8196, 0.7512, 0.8424, 0.6222],
[0.8324, 0.7431, 0.8285, 0.6727],
[0.8534, 0.7614, 0.8163, 0.8363],
[0.8534, 0.7889, 0.8244, 0.8545]]

acc=[ 0.7757,0.7566,0.7366,0.7333]

dp=[0.2202,0.1597,0.0919,0.0655]
'''
prec=[[0.8152, 0.6867, 0.7766, 0.7352],
[0.8164, 0.7, 0.7871, 0.7222],
[0.7962, 0.6829, 0.7889, 0.6222],
[0.7962, 0.6705, 0.7910, 0.5869]]
rec=[[0.9142, 0.9047, 0.9248, 0.8333],
[0.9214, 0.8888, 0.9190, 0.8666],
[0.9214, 0.8888, 0.9075, 0.9333],
[0.9214, 0.9047, 0.9190, 0.9]]

acc`=[0.7566,0.7633,0.7433,0.74]

In [ ]:
# agarwal
'''
gamma= [
[0.82199,0.715596, 0.808163,0.672727],
[0.795812,0.761468,0.804082,0.690909],
    [0.816754,0.779817,0.820408,0.727273]
[0.816754,0.752294,0.808163,0.727273]]
acc=[0.7466,0.74,0.74,0.75]

prec= [
    [0.80254, 0.70512, 0.79292, 0.64864],
    [0.81578,0.67469,0.79187,0.63157] ,
[0.80769, 0.67058, 0.78606,0.625],
[0.81410,0.68292,0.78787,0.675]]

rec= [[0.9,0.87301,0.90751,0.8],
      [0.88571,0.88888,0.90173,0.8],
      [0.9,0.90476,0.91329,0.83333]
[0.90714,0.88888,0.90173,0.9]]
dp=[0.1492,0.1131,0.0932 ,0.0895  ]    

'''



In [ ]:
#padala
'''

acc=[0.7166,0.7333,0.7266,0.69]
dp= [0.1573,0.1153,0.0867,0.0396]

prec=[[ 0.8275,0.6455, 0.7842, 0.6470],
[0.8051,0.6883,  0.7823, 0.6842],
 [0.8157, 0.6582, 0.7864, 0.6410],
[0.8201, 0.64, 0.7897, 0.6052]]

rec=[[0.8571, 0.8095, 0.8612, 0.7333],
[0.8857, 0.8412, 0.8728, 0.8666],
[0.8857, 0.8253, 0.8728, 0.8333],
 [0.8142, 0.7619, 0.8034, 0.7666]]

gamma=[[0.7591, 0.7247, 0.7755, 0.6181],
[0.8062, 0.7064, 0.7877, 0.6909],
[0.7958, 0.7247, 0.7836, 0.7090],
[0.7277, 0.6880, 0.7183, 0.6909]]

'''


individual acceptance rates
prec=[[0.8472, 0.7179, 0.8095, 0.7575],
[0.8169, 0.7236, 0.8020, 0.7027],
[0.8211, 0.7051, 0.8010, 0.6842],
[0.8478, 0.7162, 0.8228, 0.7027]]
rec=[[0.8714, 0.8888, 0.8843, 0.8333],
[0.8928, 0.8730, 0.8901, 0.8666],
[0.8857, 0.8730, 0.8843, 0.8666],
[0.8357, 0.8412, 0.8323, 0.8666]]

dp=[0.1714,0.1283,0.0996,0.0497]

acc=[0.77,0.76 0.7533,0.75]
